XS plotting
===========

This notebook uses the ``pyne.ace`` module to plot cross-sections.
ACE (A Compact ENDF) files are a point-wise representation of cross-section,
considered to be "continuous energy" (as in not discrete) for all practical
purposes.

ACE files originate from processing ENDFs (Evaluated Nuclear Data File) with
some cross section preparation code, such as NJOY.
ACE files already contain some form of approximation compared to ENDFs
(at least some form of doppler broadening, since ACE libraries are for a certain
temperature, and most likely assumptions regarding unresolved resonances and
how to represent them as points, but I would have to check more precisely to
make a strong statement), so in the future I would like to change this to using
ENDF for the plotting and making these assumptions explicit here.

In [ ]:
%matplotlib inline

latex = False

if latex:
    import matplotlib as mpl
    mpl.use("pgf")
    pgf_with_rc_fonts = {
        "font.family": "serif",
        "font.serif": [],                   # use latex default serif font
        "font.sans-serif": ["DejaVu Sans"], # use a specific sans-serif font
    }
    mpl.rcParams.update(pgf_with_rc_fonts)

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyne
from pyne import ace

mpl.style.use('seaborn')

#if not os.path.isfile("54135JEF311.ace"):
#    from urllib import urlretrieve
#    urlretrieve("https://www-nds.iaea.org/wolfram/w180/beta3/W180.ace", "W180.ace")

ene = 'Neutron energy (MeV)'

xs = '$ \sigma $ (barn)'
xs_t = '$ \sigma_{\mathrm{t}} $ (barn)'
xs_e = '$ \sigma_{\mathrm{e}} $ (barn)'
xs_a = '$ \sigma_{\mathrm{a}} $ (barn)'
xs_f = '$ \sigma_{\mathrm{f}} $ (barn)'
xs_gamma = '$ \sigma_{\gamma} $ (barn)'
xs_alpha = '$ \sigma_{\\alpha} $ (barn)'

nut = '$ \\nu_{\mathrm{t}}$'
eta = '$ \eta $ '

The main class in ``pyne.ace`` is called ``Library``. It is instantiated using the name of an ACE file, in this case one distributed with Serpent.

In [ ]:
U235 = ace.Library("/Users/rodrigo/opt/Serpent2/xsdata/jeff311/acedata/92235JEF311.ace")

One can choose to read all tables in the file or selectively read a subset by specifying an argument to the ``read`` method.

In [ ]:
U235.read()

After the call to ``read()``, the Library instance will have a dictionary called ``tables``.

In [ ]:
U235.tables

Each entry in the table corresponds to a temperature.
For example, 06c is 600 K and 12c is 1200 K.

We'll use Uranium at 1200 K, which is a reasonable temperature somewhere in the fuel pellet.

In [ ]:
U235_12 = U235.tables['92235.12c']

Once a table is selected, we can inspect the energy grid.

In [ ]:
U235_12.energy

We can also inspect the total cross section.

In [ ]:
U235_12.sigma_t


With the energy grid, and the total cross section stored on the
table, we already have basic plotting capability.

In [ ]:
fig, ax = plt.subplots()
ax.plot(U235_12.energy, U235_12.sigma_t,
        linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_t,
       title='U-235 total XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K_lin.pdf', bbox_inches='tight')

fig, ax = plt.subplots()
ax.semilogx(U235_12.energy, U235_12.sigma_t,
            linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_t,
       title='U-235 total XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K_logx.pdf', bbox_inches='tight')

fig, ax = plt.subplots()
ax.semilogy(U235_12.energy, U235_12.sigma_t,
            linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_t,
       title='U-235 total XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K_logy.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
ax.loglog(U235_12.energy, U235_12.sigma_t,
          linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_t,
       title='U-235 total XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K.pdf', bbox_inches='tight')

Here, I was going to show that the absorption cross section is also available
in the tables, but it seems like there is a bug in the absorption
cross section, since it is showing the same as the radiative capture
cross section (MT 102).

So here I will use some functionality I haven't shown yet, just to document
this bug for future reference.
Also, when the bug is eliminated, I will be aware of it.

Before making a bug report, I have to check if this is consistent in other
ACE files or if it's an isolated bug.

We'll talk about absorption cross section later then, since we can dodge the
bug and calculate it ourselves.

In [ ]:
U235_12.sigma_a

fig, ax = plt.subplots()
ax.loglog(U235_12.energy, U235_12.sigma_a,
          label='absorption',
          linewidth=0.9)
ax.loglog(U235_12.energy, U235_12.reactions[102].sigma,
          label='capture',
          linewidth=0.5)

ax.legend()
ax.set(xlabel=ene,ylabel=xs,
       title='U-235 absorption bug at 1200 K')

plt.savefig('U235_1200K_absorption_bug.pdf', bbox_inches='tight')

In [ ]:
bug = np.array_equal(U235_12.sigma_a, U235_12.reactions[102].sigma)

if bug:
    print('The bug is present, so avoid sigma_a')
else:
    print('The bug is gone. The script need to be updated')

print(pyne.__version__)

The quantity $ \nu $, which is the amount of neutrons released in a fission
event is also available in the table.

Notice that $ \nu $ has its own energy grid, which is different (smaller) than
the cross section energy grid.
This is because $ \nu $ varies relatively little, so there is no need to have
such a detailed energy grid compared to cross sections showing resonances.

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(U235_12.nu_t_energy, U235_12.nu_t_value,
            linewidth=0.5)

ax.set(xlabel=ene,ylabel=nut,
       title='U-235 neutron production at 1200 K')

plt.savefig('U235_1200K_nu.pdf', bbox_inches='tight')

To get data on a reaction, such as fission or $(n,2n)$, there is an attribute called ``reactions``.

In [ ]:
U235_12.reactions

Some important MT number for reactions are:

| MT  | reaction   | details |
| --- | ---------- | ----------- |
| 1   | n, total   | total cross |
| 2   | n0, n0'    | elastic scattering, <br />conserves kinetic energy,<br />same neutron,<br />surface interaction |
| 4   | n, n'      | inelastic scattering,<br />conserves momentum (not kinetic energy), <br />not same neutron?,<br />bulk interaction with compound nucleus formation? |
| 18  | n, fission |
| 102 | n, gamma   | AKA radiative capture |

For example, let's check the fission of U-235 at 1200 K.

In [ ]:
U235_12.reactions[18]

An instance of a Reaction contains the reaction cross section and any angular or energy distributions that may be present.

In [ ]:
U235_12_fission = U235_12.reactions[18].sigma

With the energy grid (stored on the table), and the cross section (stored on the reaction), one can generate reaction-specific plots.

In [ ]:
fig, ax = plt.subplots()
ax.loglog(U235_12.energy, U235_12_fission,
          linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_f,
       title='U-235 fission XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K_MT18.pdf', bbox_inches='tight')

I can also do a plot to project a few points of interest onto the axis.
In this plot, I marked the points at 2 MeV (neutron energy at fission),
0.065 eV (neutron energy in thermal equilibrium with a medium at aproximately 600 K),
and 0.025 eV (neutron energy in thermal equilibrium with ambient temperature).

The increase in cross section from 0.065 to 0.025 eV is very high, due to the
logarithmic nature of the plot.
This will naturally have consequences for criticality and points out to the importance
of keeping neutron absorbers when the reactor is shut down for refueling.

In [ ]:
fig, ax = plt.subplots()
ax.loglog(U235_12.energy, U235_12_fission,
          linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_f,
       title='Fission XS of U-235 at 1200 K with different neutron energies')
ax.set_ylim(ymin=0.3)

ax.axvline(0.025E-6, ymax=0.625,
           color='red',
           linewidth='0.5',
           linestyle='--')
ax.axhline(585, xmax=0.297,
           color='red',
           linewidth='0.5',
           linestyle='--')
ax.annotate('(0.025 eV, 585 barn)', (2.5E-8, 585),
            xytext=(2.9E-8, 750),
            color='red')

ax.axvline(0.065E-6, ymax=0.582,
           color='orange',
           linewidth='0.5',
           linestyle='--')
ax.axhline(335, xmax=0.327,
           color='orange',
           linewidth='0.5',
           linestyle='--')
ax.annotate('(0.065 eV, 335 barn)', (6.5E-8, 335),
            xytext=(7.1E-8, 440),
            color='orange')

ax.axvline(2, ymax=0.12,
           color='green',
           linewidth='0.5',
           linestyle='--')
ax.axhline(1.27, xmax=0.88,
           color='green',
           linewidth='0.5',
           linestyle='--')
ax.annotate('(2 MeV, 1.27 barn)', (2, 1.27),
            xytext=(0.2,2.6),
            color='green')

plt.savefig('U235_1200K_MT18_annotated.pdf', bbox_inches='tight')

It is also interesting to plot the fission cross section at different energies,
showing how doppler effect affects the fission cross section.
The integral under the curves at different energies should be the same.

In [ ]:
temperatures = [300, 600, 900, 1200, 1500, 1800]
entries = sorted(U235.tables.keys())

fig, ax = plt.subplots()
for entry, temperature in zip(entries, temperatures):
    ax.loglog(U235.tables[entry].energy, U235.tables[entry].reactions[18].sigma,
              label=f'{temperature} K',
              linewidth=0.5)

ax.legend()
ax.set_ylim(ymin=0.3)
ax.set(xlabel=ene,ylabel=xs_f,
       title='Doppler effect on U-235 fission XS')

axins = ax.inset_axes([0.05, 0.05, 0.3, 0.3])
for entry, temperature in zip(entries, temperatures):
    axins.loglog(U235.tables[entry].energy, U235.tables[entry].reactions[18].sigma,
                 linewidth=0.3)

axins.set_xlim(1.85E-6, 2.25E-6)
axins.set_ylim(1.1E1, 3E1)
for spine in axins.spines.values():
    spine.set_color('black')
    spine.set_linewidth(0.3)
axins.tick_params(axis='both',
                  which='both',
                  direction='in',
                  color='black',
                  length=2,
                  labelsize=5)
axins.tick_params(axis='x',
                  which='both',
                  labelrotation=45)

axins.set_xticklabels([1.85,1.90,1.95,2.00,2.05,2.10,2.15,2.20,2.25],minor=True)
ax.text(1.05, 0, 'x$ 10^{-6} $',
        fontsize=5,
        transform=axins.transAxes,
        horizontalalignment='left',
        verticalalignment='center')

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.savefig('U235_MT18.pdf')

In [ ]:
fig, ax = plt.subplots()
for entry, temperature in zip(entries, temperatures):
    ax.loglog(U235.tables[entry].energy, U235.tables[entry].sigma_t,
              label=f'{temperature} K',
              linewidth=0.5)

ax.legend()
ax.set_ylim(ymin=0.3)
ax.set(xlabel=ene,ylabel=xs_t,
       title='Doppler effect on U-235 total XS')

axins = ax.inset_axes([0.05, 0.05, 0.3, 0.3])
for entry, temperature in zip(entries, temperatures):
    axins.loglog(U235.tables[entry].energy, U235.tables[entry].sigma_t,
                 linewidth=0.3)

axins.set_xlim(1.85E-6, 2.25E-6)
axins.set_ylim(2.7E1, 1.1E2)
for spine in axins.spines.values():
    spine.set_color('black')
    spine.set_linewidth(0.3)
axins.tick_params(axis='both',
                  which='both',
                  direction='in',
                  color='black',
                  length=2,
                  labelsize=5)
axins.tick_params(axis='x',
                  which='both',
                  labelrotation=45)

axins.set_xticklabels([1.85,1.90,1.95,2.00,2.05,2.10,2.15,2.20,2.25],minor=True)
ax.text(1.05, 0, 'x$ 10^{-6} $',
        fontsize=5,
        transform=axins.transAxes,
        horizontalalignment='left',
        verticalalignment='center')

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.savefig('U235_MT1.pdf')

Regardless of the bug, we can make the absorption cross section roughly by
adding fission and capture cross sections.

We can also plot an insightful quantity such as the ratio of absorption to
total cross section.
Since $ \sigma_t = \sigma_a + \sigma_s $ then the complement is the probability
of scattering.

In [ ]:
U235_12_fission = U235_12.reactions[18].sigma
U235_12_capture = U235_12.reactions[102].sigma
U235_12_sigma_a = U235_12_fission + U235_12_capture

fig, ax = plt.subplots()
ax.loglog(U235_12.energy, U235_12_sigma_a,
          linewidth=0.5)

ax.set(xlabel=ene,ylabel=xs_a,
       title='U-235 absorption XS at 1200 K')
ax.set_ylim(ymin=0.3)

plt.savefig('U235_1200K_abs.pdf', bbox_inches='tight')

In [ ]:
ATR = U235_12_sigma_a / U235_12.sigma_t

fig, ax = plt.subplots()
ax.semilogx(U235_12.energy, ATR,
            linewidth=0.5)

ax.set(xlabel=ene,
       ylabel='$ \sigma_{\mathrm{a}} / \sigma_{\mathrm{t}} $',
       title='U-235 absorption to total XS ratio at 1200 K')

plt.savefig('U235_1200K_ATR.pdf', bbox_inches='tight')

In [ ]:
U235_FAR = U235_12_fission / U235_12_sigma_a

fig, ax = plt.subplots()
ax.semilogx(U235_12.energy, U235_FAR,
            linewidth=0.5)

ax.set(xlabel=ene,
       ylabel='$ \sigma_{\mathrm{f}} / \sigma_{\mathrm{a}} $ ',
       title='U-235 fission to absorption XS ratio at 1200 K')

plt.savefig('U235_1200K_FAR.pdf', bbox_inches='tight')

In [ ]:
U235_FTR = U235_12_fission / U235_12.sigma_t

fig, ax = plt.subplots()
ax.semilogx(U235_12.energy, U235_FTR,
            linewidth=0.5)

ax.set(xlabel=ene,
       ylabel='$ \sigma_{\mathrm{f}} / \sigma_{\mathrm{t}} $ ',
       title='U-235 fission to total XS ratio at 1200 K')

plt.savefig('U235_1200K_FTR.pdf', bbox_inches='tight')

We can also plot $ \eta $, which is a quantity that shows the amount of
neutrons produced per neutron absorbed.
For this we need to multiply the fission to absorption ration by $ \nu $.

Since $ \nu $ has a different energy grid, to multiply we need to
interpolate it to the cross section energy grid first.

In [ ]:
U235_nu = np.interp(U235_12.energy, U235_12.nu_t_energy, U235_12.nu_t_value)
U235_eta = U235_nu * U235_FAR

fig, ax = plt.subplots()
ax.semilogx(U235_12.energy, U235_eta,
            linewidth=0.5)

ax.set(xlabel=ene,ylabel=eta,
       title='U-235 eta at 1200 K')

plt.savefig('U235_1200K_eta.pdf', bbox_inches='tight')